In [43]:
%load_ext autoreload
%autoreload 2

from NewsContent import *
from UserContent import *
from preprocessing import *
from PEGenerator import *
import PEGenerator
from models import *
from utils import *
from Encoders import *

import os
import numpy as np
import json
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [45]:
data_root_path = "../data/Challenge/"
embedding_path = "../../"
KG_root_path = "../data/Challenge/entity"
popularity_path = "../data/Challenge/popularity"
config = {'title_length':30,
              'body_length':100,
              'max_clicked_news':50,
              'npratio':1,
              'news_encoder_name':"CNN",
              'user_encoder_name':"Att",
             'attrs':['title', 'entity', 'vert', 'image'],
             'word_filter':0,
             'data_root_path':data_root_path,
             'embedding_path':embedding_path,
             'KG_root_path':KG_root_path,
            'popularity_path':popularity_path,
            'batch_size': 32,
             'max_entity_num':5,
             'image_emb_len': 1024}
model_config = {
        'news_encoder':1,
        'popularity_user_modeling':True,
        'rel':True,
        'ctr':True,
        'content':True,
        'rece_emb':True,
        'activity':True

    }

In [46]:
News = NewsContent(config)

TrainUsers = UserContent(News.news_index,config,'train.tsv',2)
ValidUsers = UserContent(News.news_index,config,'val.tsv',1)
TestUsers = UserContent(News.news_index,config,'test.tsv',2)

24724
12678
12678


In [47]:
News.fetch_news(np.array([2]))[0]

array([   7,    8,    9,   10,   11,   12,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0, 6594,    0,    0,
          0,    0,   20,    2])

In [48]:
train_sess,train_buckets, train_user_id, train_label = get_train_input(TrainUsers.session,News.news_index,config)
test_impressions, test_userids = get_test_input(TestUsers.session,News.news_index)
val_impressions, val_userids = get_test_input(ValidUsers.session,News.news_index)

24724
24888
12678
12678


In [49]:
title_word_embedding_matrix, have_word = load_matrix(embedding_path,News.word_dict)
entity_embedding_matrix, have_word2 = load_matrix(embedding_path, News.entity_dict)

In [50]:
train_loader = DataLoader(TrainDataset(News, TrainUsers, train_sess, train_user_id, train_buckets, train_label), config['batch_size'])
val_user_data = UserDataset(News,ValidUsers)
test_user_data = UserDataset(News,TestUsers)
news_data = NewsDataset(News)

In [51]:
with open(config['data_root_path']+'image_embeddings.pkl', 'rb') as f:
    data = pickle.load(f)

    image_emb_matrix = np.zeros((len(News.news_index)+1, len(data.popitem()[1]),),dtype='float32')

    for key in data:

        if key in News.news_index:
            image_emb_matrix[News.news_index[key]] = data[key]


In [52]:
from torch.optim import Adam

model,user_encoder,news_encoder,bias_news_encoder,bias_content_scorer,scaler,time_embedding_layer,activity_gater = \
create_pe_model(config, model_config, News, title_word_embedding_matrix, entity_embedding_matrix, image_emb_matrix, device)

model.to(device)
for name, module in model.named_children():
    module.to(device)
    print(f'Moved {name} to {device}')



c:\Users\tangs\Documents\uni_stuff\RecSys\.venv\Lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Moved news_encoder to cuda
Moved bias_news_encoder to cuda
Moved pop_aware_user_encoder to cuda
Moved bias_scorer to cuda
Moved activity_gater to cuda
Moved time_embedding_layer to cuda
Moved time_distributed1 to cuda
Moved time_distributed2 to cuda
Moved time_distributed3 to cuda
Moved scaler to cuda
Moved softmax to cuda


In [53]:
colds = [5, 7, 10, 15]
topKs = 10

In [54]:
from tqdm import tqdm
val_metrics_epoch = []
num_epochs = 10
# Step 2: Create your Adam optimizer
optimizer = Adam(model.parameters(), lr=0.0001)

loss_fn = nn.CrossEntropyLoss()

# Step 3: Iterate over the data for the number of epochs
for epoch in range(num_epochs):

# Step 4: Iterate over each batch of data and compute the scores using the forward pass of the network
    model.train()
    for x, y in tqdm(train_loader):
        optimizer.zero_grad()
        x= [i.to(device) for i in x]
        y = y.to(device)
        out = model(x)
    
        # Step 5: Compute the lambda gradient values for the pairwise loss (spedup) with the compute_lambda_i method on the scores and the output labels
        loss = loss_fn(out, y)

        # Step 6: Bacward from the scores with the use of the lambda gradient values
        if loss is not None:
            # torch.autograd.backward(out, loss)
            loss.backward()
            
            # Step 7: Update the weights using the optimizer
            optimizer.step()

    model.eval()

    val_metrics = eval_model(model_config, News, user_encoder, val_impressions, val_user_data, val_userids,
               news_encoder, bias_news_encoder, activity_gater, time_embedding_layer, 
               bias_content_scorer, scaler, colds, topKs, ValidUsers, device)
    
    print("epoch: {}, metrics: {}".format(epoch, val_metrics))
    val_metrics_epoch.append(val_metrics)

    if epoch > 1:
        if (val_metrics_epoch[-1][0] < val_metrics_epoch[-2][0]):
            break



100%|██████████| 778/778 [12:01<00:00,  1.08it/s]


{5: 0, 7: 1, 10: 2, 15: 3}


100%|██████████| 12678/12678 [08:31<00:00, 24.79it/s]


epoch: 0, metrics: ([0.6926089064229985, 0.47144926379603536, 0.52643498068963, 0.5738666221515027], [[0.7247207442879775, 0.6739280855097456, 0.7121532788199455, 0.8742822128851542], [0.44844847544544264, 0.4076207803380292, 0.48691832858499523, 0.6458333333333333], [0.5142870288222771, 0.4556552139132285, 0.5905317135677942, 0.7365986575892967], [0.5418067053367344, 0.4945600973085688, 0.6037245723495729, 0.7365986575892967]], [0.00898102114991433, 0.01554145991765404, 0.020851976771516425, 0.0260843977892926, 0.030223477101861813, 0.03354274440377478, 0.036276098666792184, 0.03744753620808535, 0.03932183627415443, 0.04072756132370623], [-7.841974087817602e-09, 0.30704171685958853, 0.405508063784118, 0.4521710124183555, 0.4786200448623834, 0.49182987390848565, 0.49931879287300884, 0.5037569523158557, 0.506499656932853, 0.5080604210512404], [0.9999999921580259, 0.6140834425860614, 0.4832093457653613, 0.41906473958768725, 0.38409840499288483, 0.36444446602479236, 0.35339260694402913, 0

 95%|█████████▍| 738/778 [11:52<00:38,  1.04it/s]


KeyboardInterrupt: 

In [55]:
colds = [5, 7, 10, 15]
topKs = 10
test_metrics, test_cold_metrics, test_topics, test_ILADs, test_ILMDs = eval_model(model_config, News, user_encoder, test_impressions, test_user_data, test_userids,
               news_encoder, bias_news_encoder, activity_gater, time_embedding_layer, 
               bias_content_scorer, scaler, colds, topKs, TestUsers, device)

{5: 0, 7: 1, 10: 2, 15: 3}


100%|██████████| 12678/12678 [09:56<00:00, 21.26it/s]


In [ ]:
print("val metrics", val_metrics_epoch)
print("test metrics", test_metrics)
print("cold", test_cold_metrics)
print("diversity", [test_ILADs, test_ILMDs])

results = {"config": config,
           "model_config" : model_config,
            "val metrics": val_metrics_epoch, 
           "test metrics": test_metrics,
           "cold": test_cold_metrics,
           "diversity": [test_ILADs, test_ILMDs]}

import json
with open('results{}.json'.format(time.time()), 'w') as f:
    json.dump(results, f)

val metrics []
test metrics [0.6634010468600947, 0.4522433731561524, 0.5000726062253267, 0.554345353995634]
cold [[0.6631188748709034, 0.6594726072544687, 0.548941892321481, 0.6636733228644994], [0.42642099280030316, 0.3334796925342455, 0.3155838143338143, 0.45982142857142855], [0.4664161142940799, 0.3930518848560765, 0.359610683584375, 0.5115775579644578], [0.5256374857231534, 0.4483332575556277, 0.42945446810269333, 0.5673416717378086]]
diversity [[-7.851376934445681e-09, 0.3058020994337524, 0.40556908791317964, 0.453821723018089, 0.4803567541277471, 0.49424605803791327, 0.5020613400252352, 0.5062891028011648, 0.5087711844539807, 0.5102065742006435], [0.9999999921486231, 0.6116042091166076, 0.5191388283289838, 0.47957104162888525, 0.4561223003799973, 0.44408288730875817, 0.43646860975518853, 0.43145813391324866, 0.4280776474463261, 0.4259116095988076]]


## Finished
## ---